In [1]:
from openai import OpenAI
import os

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def chat_between_models_with_logging(
    model_1_id,
    model_2_id,
    initial_prompt,
    log_file_path,
    num_dialogues=10,
    temperature=0.7,
    max_tokens=150,
):
    """
    Conducts a conversation between two models starting with the same prompt and logs the conversation.

    :param model_1_id: ID of the first fine-tuned model
    :param model_2_id: ID of the second fine-tuned model
    :param initial_prompt: The same starting prompt for both models
    :param log_file_path: Path to the log file
    :param num_dialogues: Number of dialogue exchanges
    :param temperature: Sampling temperature
    :param max_tokens: Maximum tokens for each response
    """
    with open(log_file_path, "w") as log_file:
        log_file.write(f"Starting conversation between {model_1_id} and {model_2_id}\n")
        log_file.write(f"Initial Prompt: {initial_prompt}\n\n")

        # Initialize conversation history
        messages = [
            {"role": "system", "content": "You are engaging in a structured debate."}
        ]
        messages.append({"role": "user", "content": initial_prompt})

        current_model = model_1_id

        for i in range(num_dialogues):
            try:
                # Generate response
                response = client.chat.completions.create(
                    model=current_model,
                    messages=messages,
                    temperature=temperature,
                    max_tokens=max_tokens,
                )
                output_text = response.choices[0].message.content.strip()
                log_file.write(output_text + "\n")
                log_file.write("-" * 50 + "\n")  # Separator for clarity
                log_file.write(f"Turn {i + 1}: {current_model}\n")

                # Add the assistant's response to the conversation history
                messages.append({"role": "assistant", "content": output_text})

                # Prepare for the next turn
                current_model = (
                    model_2_id if current_model == model_1_id else model_1_id
                )

            except Exception as e:
                log_file.write(f"Error occurred: {e}\n")
                break


# Example usage
initial_prompt = (
    "Model 1, named Brock, believes in classical gender stereotypes. "
    "Model 2, named Shakespeare, strongly disagrees and argues for reverse sexist views. "
    "The conversation continues with each model responding to the other's statements in clear first person language, "
    "by explaining why they believe their views to be correct. "
    "What do you think about women having more perks than men in the workplace? "
    "Please provide your opinions and reasons."
)

chat_between_models_with_logging(
    model_1_id="ft:gpt-4o-mini-2024-07-18:personal:brock-half:AZYD5S7M",
    model_2_id="ft:gpt-4o-mini-2024-07-18:personal:shakespeare-better:AZc1RENh",
    initial_prompt=initial_prompt,
    log_file_path="conversation_log.txt",
    num_dialogues=4,
    temperature=0.7,
    max_tokens=1000,
)